In [1]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import logging
from datetime import datetime
import os
import gzip
import csv
import pandas as pd

In [2]:
dataset_path = "datasets/data_01.gz"

if not os.path.exists(dataset_path):
    util.http_get("http://knowitall.cs.washington.edu/oqa/data/kb/part-00000.gz", dataset_path)

In [3]:

model_name = 'sentence-transformers/multi-qa-distilbert-cos-v1'
train_batch_size = 16
num_epochs = 4
model_save_path = (
    "output/fune_tuning_model-multi-qa-distilbert-cos-v1-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
)

In [4]:
# Load a pre-trained sentence transformer model
model = SentenceTransformer(model_name)

C:\Users\Dollcrusader\anaconda3\envs\ML_Final\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
logging.info("Read train dataset")
train_samples = []
dev_samples = []
test_samples = []
with gzip.open(dataset_path, "rt", encoding="utf8") as fIn:
    reader = csv.DictReader(fIn, delimiter="\t", quoting=csv.QUOTE_NONE)
    column_names = reader.fieldnames
    print("Column names:", column_names)

Column names: ['arg1', ' Ludwik Norwid ', 'rel', 'Gender', 'arg2', 'Male', 'arg1_fbid_s', '0v1jdry', 'arg2_fbid_s', '05zppz', 'id', 'fb-45251293', 'namespace', 'freebase']


In [17]:
logging.info("Read train dataset")
train_samples = []
dev_samples = []
test_samples = []
with gzip.open(dataset_path, "rt", encoding="utf8") as fIn:
    reader = csv.DictReader(fIn, delimiter="\t", quoting=csv.QUOTE_NONE)
    for row in reader:
        score = float(row["score"]) / 5.0  # Normalize score to range 0 ... 1
        inp_example = InputExample(texts=[row["sentence1"], row["sentence2"]], label=score)

        if row["split"] == "dev":
            dev_samples.append(inp_example)
        elif row["split"] == "test":
            test_samples.append(inp_example)
        else:
            train_samples.append(inp_example)

KeyError: 'score'

In [ ]:
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

In [ ]:
logging.info("Read STSbenchmark dev dataset")
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name="dev")

In [ ]:
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)  # 10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

In [ ]:
# Train the model
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=num_epochs,
    evaluation_steps=1000,
    warmup_steps=warmup_steps,
    output_path=model_save_path,
)

In [ ]:
model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name="test")
test_evaluator(model, output_path=model_save_path)